In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
!ls

%cd ./ResUNetplusplus
!ls 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
ResUNetplusplus
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing/ResUNetplusplus
'=1.0.7,'			   process_image_sample.py
'Copy of execution_script.ipynb'   __pycache__
 data				   README.md
 data_generator.py		   result
 deployment_model.py		   results.png
 execution_script.ipynb		   resume_training.py
 files				   Resunet++.png
 frozen_models			   resunet_pp_modified.py
 GFLOPs.ipynb			   resunet.py
 infer.py			   run_miouTesting.py
 metrics.py			   run_modified.py
 m_resunet_modified.py		   run.py
 m_resunet.py			   run_sample.py
 new_data			   segmentation_models.ipynb
 process_image_deployment.py	   sm
 process_image.py		   unet.py


In [3]:
# !pip install git+https://github.com/qubvel/segmentation_models
# !pip install tensorflow==2.2.0
# !pip install -U segmentation-models==0.2.1 #install this
!pip install keras==2.3.1 #install this
# !pip install tensorflow==2.2.0 #install this
!pip install tensorflow==2.1 # For TPU training only
### Then restart runtime
# !pip install keras_applications >=1.0.7,<=1.0.8
!pip install segmentation-models==1.0.1

     |████████████████████████████████| 378kB 6.8MB/s 
     |████████████████████████████████| 51kB 3.6MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 25.8MB/s 
     |████████████████████████████████| 3.9MB 39.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=761f80e4bde315d8a988af7edafb656f091b3c75f8cf96bfb58afed7106ec088
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    

In [2]:
import segmentation_models as sm

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [8]:
# from tensorflow import keras
model = sm.Unet('mobilenet', classes=1, activation='softmax')
model.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


17227776/17225924 [==============================] - 0s 0us/step
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 3 864         conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 3 128         conv1[0][0]                      
___________________________

In [4]:
!pip install cloud-tpu-client
# https://github.com/tensorflow/tensorflow/issues/40622

import tensorflow as tf
from cloud_tpu_client import Client
print(tf.__version__)

Client().configure_tpu_version(tf.__version__, restart_type='ifNeeded')

     |████████████████████████████████| 61kB 3.5MB/s 
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
2.1.0


In [ ]:
#!python3 process_image.py

Total Size:  1000
Training Size:  800
Validation Size:  100
Testing Size:  100
100% 100/100 [00:03<00:00, 26.61it/s]
100% 100/100 [00:03<00:00, 29.23it/s]
  4% 30/800 [02:09<3:22:54, 15.81s/it]

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from data_generator import DataGen
from unet import Unet
from resunet import ResUnet
# from m_resunet_modified import ResUnetPlusPlus
from resunet_pp_modified import ResUnetPlusPlus
from metrics import dice_coef, dice_loss, miou_coef, miou_loss


if __name__ == "__main__":
    ## Path
    file_path = "files/"
    model_path = "files/unet_resnet_miou.h5"
    model_name = "unet_resnet_miou"

    ## Create files folder
    try:
        os.mkdir("files")
    except:
        pass

    train_path = "new_data/Kvasir-SEG/train/"
    valid_path = "new_data/Kvasir-SEG/valid/"

    ## Training
    train_image_paths = glob(os.path.join(train_path, "images", "*"))
    train_mask_paths = glob(os.path.join(train_path, "masks", "*"))
    train_image_paths.sort()
    train_mask_paths.sort()

    # train_image_paths = train_image_paths[:2000]
    # train_mask_paths = train_mask_paths[:2000]

    ## Validation
    valid_image_paths = glob(os.path.join(valid_path, "images", "*"))
    valid_mask_paths = glob(os.path.join(valid_path, "masks", "*"))
    valid_image_paths.sort()
    valid_mask_paths.sort()

    ## Parameters
    image_size = 256
    batch_size = 8
    lr = 1e-4
    epochs = 10

    train_steps = len(train_image_paths)//batch_size
    valid_steps = len(valid_image_paths)//batch_size

    print("train steps:", train_steps , "valid_steps", valid_steps)

    ## Generator
    train_gen = DataGen(image_size, train_image_paths, train_mask_paths, batch_size=batch_size)
    valid_gen = DataGen(image_size, valid_image_paths, valid_mask_paths, batch_size=batch_size)

    ## Unet
    # arch = Unet(input_size=image_size)
    # model = arch.build_model()
    model = sm.Unet('seresnet18', classes=1, activation='sigmoid'
    # , encoder_weights='imagenet'
    )

    ## ResUnet
    # arch = ResUnet(input_size=image_size)
    # model = arch.build_model()

    ## ResUnet++
    # arch = ResUnetPlusPlus(input_size=image_size)
    # model = arch.build_model()

    optimizer = Nadam(lr)
    # metrics = [Recall(), Precision(), dice_coef, MeanIoU(num_classes=2)]
    # model.compile(loss=dice_loss, optimizer=optimizer, metrics=metrics)
    metrics = [Recall(), Precision(), dice_coef, MeanIoU(num_classes=2),miou_coef]
    # model.compile(loss=miou_loss, optimizer=optimizer, metrics=metrics)
    model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
    )
    # model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=optimizer, metrics=metrics)

    csv_logger = CSVLogger(f"{file_path}{model_name}_{batch_size}_{epochs}.csv", append=False)
    checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
    callbacks = [csv_logger, checkpoint, reduce_lr, early_stopping]
   
    model.fit(
        train_gen,
        validation_data=valid_gen,
        epochs=100,
        )
    # model.fit_generator(train_gen,
    #         validation_data=valid_gen,
    #         steps_per_epoch=train_steps,
    #         validation_steps=valid_steps,
    #         epochs=epochs,
    #         callbacks=callbacks)
    
# !python3 run_modified.py

KeyboardInterrupt: ignored

In [9]:
## TPU Setup



import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from data_generator import DataGen
from unet import Unet
from resunet import ResUnet
# from m_resunet_modified import ResUnetPlusPlus
from resunet_pp_modified import ResUnetPlusPlus
from metrics import dice_coef, dice_loss, miou_coef, miou_loss


# Distribution strategies
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)




INFO:tensorflow:Initializing the TPU system: 10.4.156.130:8470


INFO:tensorflow:Initializing the TPU system: 10.4.156.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

if __name__ == "__main__":
    ## Path
    file_path = "sm/files/"
    model_path = "sm/files/unet_resnet_miou.h5"
    model_name = "unet_resnet_miou"

    ## Create files folder
    try:
        os.mkdir("files")
    except:
        pass

    train_path = "new_data/Kvasir-SEG/train/"
    valid_path = "new_data/Kvasir-SEG/valid/"

    ## Training
    train_image_paths = glob(os.path.join(train_path, "images", "*"))
    train_mask_paths = glob(os.path.join(train_path, "masks", "*"))
    train_image_paths.sort()
    train_mask_paths.sort()

    # train_image_paths = train_image_paths[:2000]
    # train_mask_paths = train_mask_paths[:2000]

    ## Validation
    valid_image_paths = glob(os.path.join(valid_path, "images", "*"))
    valid_mask_paths = glob(os.path.join(valid_path, "masks", "*"))
    valid_image_paths.sort()
    valid_mask_paths.sort()

    ## Parameters
    image_size = 256
    batch_size = 8
    lr = 1e-4
    epochs = 10

    train_steps = len(train_image_paths)//batch_size
    valid_steps = len(valid_image_paths)//batch_size

    print("train steps:", train_steps , "valid_steps", valid_steps)

    ## Generator
    train_gen = DataGen(image_size, train_image_paths, train_mask_paths, batch_size=batch_size)
    valid_gen = DataGen(image_size, valid_image_paths, valid_mask_paths, batch_size=batch_size)

    loss = sm.losses.bce_jaccard_loss
    metrics = [sm.metrics.iou_score]

    strategy = tf.distribute.experimental.TPUStrategy(resolver)
    with strategy.scope():
      if os.path.exists(model_path):
        # with CustomObjectScope({'loss': sm.losses.bce_jaccard_loss, 'metrics': sm.metrics.iou_score}):
        model = load_model(model_path, custom_objects={'loss': loss , 'metrics':metrics})
      else:
        
        model = sm.Unet('seresnet18', classes=1, activation='sigmoid')
        model.compile('Adam',loss=loss,    metrics=metrics,    )
    
    csv_logger = CSVLogger(f"{file_path}{model_name}_{batch_size}_{epochs}.csv", append=False)
    checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
    callbacks = [csv_logger, checkpoint, reduce_lr, early_stopping]
   
    model.fit_generator(
        train_gen,
        validation_data=valid_gen,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        epochs=50,
        callbacks=callbacks
        )
    
# !python3 run_modified.py

train steps: 3100 valid_steps 12
INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Epoch 1/50
3100/3100 [==============================] - 11910s 4s/step - loss: 0.7919 - iou_score: 0.5031 - val_loss: 1.4592 - val_iou_score: 0.4954

Epoch 00001: val_loss improved from inf to 1.45919, saving model to sm/files/unet_resnet_miou.h5
Epoch 2/50
3100/3100 [==============================] - 4783s 2s/step - loss: 0.5446 - iou_score: 0.6628 - val_loss: 0.5091 - val_iou_score: 0.6905

Epoch 00002: val_loss improved from 1.45919 to 0.50910, saving model to sm/files/unet_resnet_miou.h5
Epoch 3/50
3100/3100 [==============================] - 4783s 2s/step - loss: 0.4472 - iou_score: 0.7200 - val_loss: 1.7610 - val_iou_score: 0.5420

Epoch 00003: val_loss did not improve from 0.50910
Epoch 4/50
3100/3100 [==============================] - 4762s 2s/step - loss: 0.3807 - iou_score: 0.7612 - val_loss: 0.2154 - val_iou_score: 0.6686

Epoch 00004: val_loss improved from 0.50910 to 0.21536, saving model to sm/files/unet_resnet_miou.h5
Epoch 5/50
3100/3100 [==============================]

In [ ]:
%%writefile sm/run_sm.py
## TPU Setup

import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from data_generator import DataGen
from unet import Unet
from resunet import ResUnet
# from m_resunet_modified import ResUnetPlusPlus
from resunet_pp_modified import ResUnetPlusPlus
from metrics import dice_coef, dice_loss, miou_coef, miou_loss


# Distribution strategies
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

if __name__ == "__main__":
    ## Path
    file_path = "sm/files/"
    model_path = "sm/files/unet_resnet_miou.h5"
    model_name = "unet_resnet_miou"

    ## Create files folder
    try:
        os.mkdir("files")
    except:
        pass

    train_path = "new_data/Kvasir-SEG/train/"
    valid_path = "new_data/Kvasir-SEG/valid/"

    ## Training
    train_image_paths = glob(os.path.join(train_path, "images", "*"))
    train_mask_paths = glob(os.path.join(train_path, "masks", "*"))
    train_image_paths.sort()
    train_mask_paths.sort()

    # train_image_paths = train_image_paths[:2000]
    # train_mask_paths = train_mask_paths[:2000]

    ## Validation
    valid_image_paths = glob(os.path.join(valid_path, "images", "*"))
    valid_mask_paths = glob(os.path.join(valid_path, "masks", "*"))
    valid_image_paths.sort()
    valid_mask_paths.sort()

    ## Parameters
    image_size = 256
    batch_size = 8
    lr = 1e-4
    epochs = 10

    train_steps = len(train_image_paths)//batch_size
    valid_steps = len(valid_image_paths)//batch_size

    print("train steps:", train_steps , "valid_steps", valid_steps)

    ## Generator
    train_gen = DataGen(image_size, train_image_paths, train_mask_paths, batch_size=batch_size)
    valid_gen = DataGen(image_size, valid_image_paths, valid_mask_paths, batch_size=batch_size)

    loss = sm.losses.bce_jaccard_loss
    metrics = [sm.metrics.iou_score]

    strategy = tf.distribute.experimental.TPUStrategy(resolver)
    with strategy.scope():
      if os.path.exists(model_path):
        # with CustomObjectScope({'loss': sm.losses.bce_jaccard_loss, 'metrics': sm.metrics.iou_score}):
        model = load_model(model_path, custom_objects={'loss': loss , 'metrics':metrics})
      else:
        
        model = sm.Unet('seresnet18', classes=1, activation='sigmoid')
        model.compile('Adam',loss=loss,    metrics=metrics,    )
    
    csv_logger = CSVLogger(f"{file_path}{model_name}_{batch_size}_{epochs}.csv", append=False)
    checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
    callbacks = [csv_logger, checkpoint, reduce_lr, early_stopping]
   
    model.fit_generator(
        train_gen,
        validation_data=valid_gen,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        epochs=50,
        callbacks=callbacks
        )
    
# !python3 run_modified.py

Overwriting sm/run_sm.py


In [ ]:
!python3 sm/run_sm.py

2020-12-14 06:33:43.244824: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-12-14 06:33:43.244961: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-12-14 06:33:43.244981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Traceback (most recent call last):
  File "sm/run_sm.py", line 12, in <module>
    from data_generator import DataGen
ModuleNotFoundError: No module name

In [ ]:
%pycat sm/run_sm.py

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

if __name__ == "__main__":
    ## Path
    file_path = "sm/files/"
    model_path = "sm/files/unet_resnet_miou.h5"
    model_name = "unet_resnet_miou"

    ## Create files folder
    try:
        os.mkdir("files")
    except:
        pass

    train_path = "new_data/Kvasir-SEG/train/"
    valid_path = "new_data/Kvasir-SEG/valid/"

    ## Training
    train_image_paths = glob(os.path.join(train_path, "images", "*"))
    train_mask_paths = glob(os.path.join(train_path, "masks", "*"))
    train_image_paths.sort()
    train_mask_paths.sort()

    # train_image_paths = train_image_paths[:2000]
    # train_mask_paths = train_mask_paths[:2000]

    ## Validation
    valid_image_paths = glob(os.path.join(valid_path, "images", "*"))
    valid_mask_paths = glob(os.path.join(valid_path, "masks", "*"))
    valid_image_paths.sort()
    valid_mask_paths.sort()

    ## Parameters
    image_size = 256
    batch_size = 8
    lr = 1e-4
    epochs = 10

    train_steps = len(train_image_paths)//batch_size
    valid_steps = len(valid_image_paths)//batch_size

    print("train steps:", train_steps , "valid_steps", valid_steps)

    ## Generator
    train_gen = DataGen(image_size, train_image_paths, train_mask_paths, batch_size=batch_size)
    valid_gen = DataGen(image_size, valid_image_paths, valid_mask_paths, batch_size=batch_size)

    loss = sm.losses.bce_jaccard_loss
    metrics = [sm.metrics.iou_score]

    strategy = tf.distribute.experimental.TPUStrategy(resolver)
    with strategy.scope():
      if os.path.exists(model_path):
        # with CustomObjectScope({'loss': sm.losses.bce_jaccard_loss, 'metrics': sm.metrics.iou_score}):
        # custom_objects={'loss': loss , 'metrics':metrics},
        model = load_model(model_path, compile=False)
        model.compile('Adam',loss=loss,    metrics=metrics,    )
      else:
        model = sm.Unet('seresnet18', classes=1, activatNion='sigmoid')
        model.compile('Adam',loss=loss,    metrics=metrics,    )
    
    csv_logger = CSVLogger(f"{file_path}{model_name}_{batch_size}_{epochs}.csv", append=False)
    checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
    callbacks = [csv_logger, checkpoint, reduce_lr, early_stopping]
   
    model.fit_generator(
        train_gen,
        validation_data=valid_gen,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        epochs=50,
        callbacks=callbacks
        )
    
# !python3 run_modified.py

train steps: 3100 valid_steps 12
INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Epoch 1/50
1011/3100 [========>.....................] - ETA: 2:20:49 - loss: 0.3353 - iou_score: 0.7882